<a href="https://colab.research.google.com/github/echung2/echung2/blob/master/Hanna_Ren_Frank_Prob20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install GiNZA, an open Source Japanese NLP library, which is not pre-installed on Google Colab
!pip install -U ginza ja_ginza


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 49.0 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import spacy
import ginza
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
from google.colab import files
import re
import csv

In [27]:
# Upload file
uploaded = files.upload()

Saving H_Frankenstein_OCR.txt to H_Frankenstein_OCR.txt


In [28]:
# Load text from uploaded file
file_name = list(uploaded.keys())[0]
with open(file_name, 'r', encoding='utf-8') as file:
    text = file.read()

In [29]:
# Preprocess text: remove special characters
text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces

In [30]:
# Define Japanese stopwords
stopwords = set([
    'Page', 'は', 'が', 'の', 'に', 'を', 'へ', 'と', 'で', 'や', 'も', 'から', 'まで', 'だけ', 'より', 'しか', 'ながら', 'など',
    '私', '僕', '俺', 'あなた', '君', '彼', '彼女', '私たち', 'あなたたち', '皆', 'これ', 'それ', 'あれ', 'どれ', 'ここ',
    'そこ', 'あそこ', 'どこ', 'とても', 'かなり', '少し', 'ちょっと', '大体', 'ほとんど', 'また', 'なぜ', 'どう', '何', 'いつ',
    'どこ', 'ただ', 'ようやく', 'やはり', 'さらに', 'すぐに', 'いる', 'ある', 'する', 'なる', '行く', '来る', '見る', '聞く',
    '言う', '思う', '感じる', '分かる', '知る', '良い', '悪い', '大きい', '小さい', '高い', '低い', '多い', '少ない', '簡単だ',
    '難しい', '長い', '短い', 'そして', 'しかし', 'それでも', 'だから', 'または', 'それに', 'つまり', 'なぜなら', 'ああ', 'うん',
    'ええ', 'はい', 'いいえ', 'おお', 'あれ', 'まあ', 'えっ', 'あっ', 'ふうん', 'こと', 'もの', 'ところ', 'よう', '例え',
    '例えば', '場合', 'ような', '様子', 'する', 'ある', 'いる', 'なる', 'れる', 'の', 'も', 'た', 'が', 'に', 'は', 'を', 'で'
])

In [32]:
# Load the "ja_ginza" SpaCy pipeline
try:
    nlp = spacy.load("ja_ginza")
except Exception as e:
    print(f"Error loading ja_ginza: {e}")

In [33]:
# Split text into chunks if necessary
chunk_size = 10
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

In [34]:
# Tokenize and filter words
tokens = []
for chunk in chunks:
    doc = nlp(chunk)
    for token in doc:
        if token.pos_ in ['NOUN', 'VERB', 'ADJ'] and token.text not in stopwords:
            tokens.append(token.text)

In [18]:
# Display extracted tokens for verification (optional)
print(f"Filtered Tokens: {tokens[:50]}")

Filtered Tokens: ['羽', '贈る', '拳銃', '羽', '贈る', '拳銃', 'い', 'つ', 'ちょう', '手', '中', '銃', 'つめ', 'く', '光る', '黒色', '銃', 'きみ', '小さな', '手のひら', '慣', '真', '黒', '重み', '戸惑い', '汗', '真夜中', '忍び込ん', '書斎', '無数', '傷', '刻', 'ま', '年', '代物', '机', '色', '褪せ', '木目', '怪物', '眼', '球', '怯え', '机', '引き出し', '刃先', '錆び', 'メ', 'IMRI', '用']


In [19]:
# Create dictionary and corpus for LDA
dictionary = corpora.Dictionary([tokens])
corpus = [dictionary.doc2bow(tokens)]

In [20]:
# Apply LDA for topic modeling with additional parameters
num_topics = 20  # Specify the number of topics
lda_model = LdaModel(
    corpus=corpus,
    num_topics=num_topics,
    id2word=dictionary,
    passes=20,
    iterations=400,
    random_state=50
)

In [21]:
# Extract topics and words with probabilities
topics = lda_model.show_topics(num_topics=num_topics, num_words=20, formatted=False)

In [22]:
# Prepare data for CSV
csv_data = []
for topic_idx, topic_words in topics:
    topic_row = [f"Topic {topic_idx + 1}"]
    for word, prob in topic_words:
        topic_row.extend([word, f"{prob:.4f}"])  # Add Word and Weight alternately
    csv_data.append(topic_row)

In [25]:
# Save the topics to a CSV file with UTF-8 with BOM encoding
output_file = "output_mayaha_prob_20_words.csv"
with open(output_file, mode='w', encoding='utf-8-sig', newline='') as file:  # 'utf-8-sig' for Japanese support
    writer = csv.writer(file)
    # Write header
    header = ["Topic"] + [f"Word{i//2 + 1}" if i % 2 == 0 else f"Weight{i//2 + 1}" for i in range(40)]
    writer.writerow(header)
    # Write topics and words with weights
    for row in csv_data:
        writer.writerow(row)

print(f"Results saved to {output_file}")

Results saved to output_mayaha_prob_20_words.csv


In [26]:
# Download the output file if running in Colab
from google.colab import files
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>